<a href="https://colab.research.google.com/github/Harris-giki/AI_VoiceAssistants/blob/main/Silero_Gemini_Kokoro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install dependencies
!pip install -q torchaudio transformers google-generativeai
!pip install -q kokoro>=0.9.2 soundfile
!pip install -q silero-vad
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.7 MB/s eta 0:00:00


In [ ]:
# Step 2: Import Libraries
import torch
from transformers import pipeline as hf_pipeline
from google.generativeai import configure, GenerativeModel
from kokoro import KPipeline
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps
import soundfile as sf
from IPython.display import Audio, display
from google.colab import files
import os

In [ ]:
# Step 3: Upload audio
print("Upload a WAV file for speech detection + transcription...")
uploaded = files.upload()
audio_path = list(uploaded.keys())[0]

Upload a WAV file for speech detection + transcription...


Saving testing-english (online-audio-converter.com).wav to testing-english (online-audio-converter.com) (1).wav


In [ ]:
# Step 4: Use Silero VAD to isolate speech segments
model = load_silero_vad()
wav = read_audio(audio_path)

speech_timestamps = get_speech_timestamps(wav, model, return_seconds=True)
print("Speech detected at:", speech_timestamps)

Speech detected at: [{'start': 0.6, 'end': 1.1}, {'start': 1.4, 'end': 6.0}, {'start': 6.5, 'end': 11.6}]


In [ ]:
# Step 5: Whisper STT
asr_pipeline = hf_pipeline("automatic-speech-recognition", model="openai/whisper-small")

# Transcribe the uploaded audio file
transcription_result = asr_pipeline(audio_path)
transcription = transcription_result["text"]
print("Transcription:", transcription)

config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


Transcription:  Hey, I want to know which type of LLM am I talking to. Also tell me what is the capital of Pakistan. Tell me any five things about the country as well.


In [ ]:
# Step 6: Gemini Response
GEMINI_API_KEY = "add_api"
configure(api_key=GEMINI_API_KEY)

model = GenerativeModel("gemini-2.0-flash")
response = model.generate_content(transcription).text
print("Gemini Response:", response)

Gemini Response: I am a large language model, trained by Google.

The capital of Pakistan is Islamabad.

Here are five things about Pakistan:

1.  **Geography:** Pakistan is geographically diverse, with landscapes ranging from deserts and plains to forests, hills, and plateaus. It is home to some of the world's highest mountain ranges, including the Himalayas, Karakoram, and Hindu Kush.
2.  **Indus Valley Civilization:** Pakistan is the site of one of the world's oldest civilizations, the Indus Valley Civilization, which flourished around 2500-1900 BCE.
3.  **Languages:** While Urdu is the national language, English is widely used in business and education. Other major regional languages include Punjabi, Sindhi, Pashto, and Balochi.
4.  **Cricket:** Cricket is the most popular sport in Pakistan, and the national team has won the Cricket World Cup once.
5.  **Islamic Republic:** Pakistan is an Islamic republic with a parliamentary system of government. Islam plays a significant role in 

In [ ]:
# Step 7: Kokoro TTS
tts_pipeline = KPipeline(lang_code='a')
generator = tts_pipeline(response, voice='af_heart')

for i, (gs, ps, audio) in enumerate(generator):
    sf.write(f'response_{i}.wav', audio, 24000)
    display(Audio(data=audio, rate=24000, autoplay=True))

config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)


kokoro-v1_0.pth:   0%|          | 0.00/327M [00:00<?, ?B/s]

af_heart.pt:   0%|          | 0.00/523k [00:00<?, ?B/s]

KeyboardInterrupt: 